# Exercício 1 (Dilithium) - Trabalho Prático 3

**Grupo 6:** 


Ruben Silva - pg57900

Luís Costa - pg55970

# Resolução do Dilithium
[Repositório Educativo](https://github.com/GiacomoPope/dilithium-py)

[Material de Estruturas Criptográficas - Universidade Minho](https://www.dropbox.com/scl/fo/1xckq5unht6ky6xftc91b/AEfAXCq0CYMZJuFR5QQo_mc?rlkey=3vvacqrzmdjhkfs451u4u8xxi&e=2&st=baplvnrk&dl=0)

### imports

In [218]:
import os
import random
from xoflib import shake128, shake256
from Crypto.Cipher import AES
from typing import Optional

## Funções Auxiliares

In [219]:
def reduce_mod_pm(x, n):
    x = x % n
    if x > (n >> 1):
        x -= n
    return x


def decompose(r, a, q):
    rp = r % q
    r0 = reduce_mod_pm(rp, a)
    r1 = rp - r0
    if rp - r0 == q - 1:
        r1 = 0
        r0 = r0 - 1
    else:
        r1 = (rp - r0) // a
    return r1, r0


def high_bits(r, a, q):
    r1, _ = decompose(r, a, q)
    return r1


def low_bits(r, a, q):
    _, r0 = decompose(r, a, q)
    return r0


def make_hint(z, r, a, q):
    r1 = high_bits(r, a, q)
    v1 = high_bits(r + z, a, q)
    return int(r1 != v1)


def make_hint_optimised(z0, r1, a, q):
    gamma2 = a >> 1
    if z0 <= gamma2 or z0 > (q - gamma2) or (z0 == (q - gamma2) and r1 == 0):
        return 0
    return 1


def use_hint(h, r, a, q):
    m = (q - 1) // a
    r1, r0 = decompose(r, a, q)
    if h == 1:
        if r0 > 0:
            return (r1 + 1) % m
        return (r1 - 1) % m
    return r1


def check_norm_bound(n, b, q):
    x = n % q
    x = ((q - 1) >> 1) - x
    x = x ^ (x >> 31)
    x = ((q - 1) >> 1) - x
    return x >= b


def xor_bytes(a, b):
    return bytes(a ^ b for a, b in zip(a, b))


## Class Anel Polinomial

In [220]:
class PolynomialRing:
    def __init__(self, q, n):
        self.q = q
        self.n = n
        self.element = Polynomial

    def gen(self):
        return self([0, 1])

    def random_element(self):
        coefficients = [random.randint(0, self.q - 1) for _ in range(self.n)]
        return self(coefficients)

    def __call__(self, coefficients):
        if isinstance(coefficients, int):
            return self.element(self, [coefficients])
        if not isinstance(coefficients, list):
            raise TypeError(
                f"Polynomials should be constructed from a list of integers, of length at most d = {self.n}"
            )
        return self.element(self, coefficients)

    def __repr__(self):
        return f"Univariate Polynomial Ring in x over Finite Field of size {self.q} with modulus x^{self.n} + 1"


class Polynomial:
    def __init__(self, parent, coefficients):
        self.parent = parent
        self.coeffs = self._parse_coefficients(coefficients)

    def is_zero(self):
        return all(c == 0 for c in self.coeffs)

    def is_constant(self):
        return all(c == 0 for c in self.coeffs[1:])

    def _parse_coefficients(self, coefficients):
        l = len(coefficients)
        if l > self.parent.n:
            raise ValueError(
                f"Coefficients describe polynomial of degree greater than maximum degree {self.parent.n}"
            )
        elif l < self.parent.n:
            coefficients = coefficients + [0 for _ in range(self.parent.n - l)]
        return coefficients

    def reduce_coefficients(self):
        self.coeffs = [c % self.parent.q for c in self.coeffs]
        return self

    def _add_mod_q(self, x, y):
        return (x + y) % self.parent.q

    def _sub_mod_q(self, x, y):
        return (x - y) % self.parent.q

    def _schoolbook_multiplication(self, other):
        n = self.parent.n
        a = self.coeffs
        b = other.coeffs
        new_coeffs = [0 for _ in range(n)]
        for i in range(n):
            for j in range(0, n - i):
                new_coeffs[i + j] += a[i] * b[j]
        for j in range(1, n):
            for i in range(n - j, n):
                new_coeffs[i + j - n] -= a[i] * b[j]
        return [c % self.parent.q for c in new_coeffs]

    def __neg__(self):
        neg_coeffs = [(-x % self.parent.q) for x in self.coeffs]
        return self.parent(neg_coeffs)

    def _add_(self, other):
        if isinstance(other, type(self)):
            new_coeffs = [
                self._add_mod_q(x, y) for x, y in zip(self.coeffs, other.coeffs)
            ]
        elif isinstance(other, int):
            new_coeffs = self.coeffs.copy()
            new_coeffs[0] = self._add_mod_q(new_coeffs[0], other)
        else:
            raise NotImplementedError("Polynomials can only be added to each other")
        return new_coeffs

    def __add__(self, other):
        new_coeffs = self._add_(other)
        return self.parent(new_coeffs)

    def __radd__(self, other):
        return self.__add__(other)

    def __iadd__(self, other):
        self = self + other
        return self

    def _sub_(self, other):
        if isinstance(other, type(self)):
            new_coeffs = [
                self._sub_mod_q(x, y) for x, y in zip(self.coeffs, other.coeffs)
            ]
        elif isinstance(other, int):
            new_coeffs = self.coeffs.copy()
            new_coeffs[0] = self._sub_mod_q(new_coeffs[0], other)
        else:
            raise NotImplementedError(
                "Polynomials can only be subtracted from each other"
            )
        return new_coeffs

    def __sub__(self, other):
        new_coeffs = self._sub_(other)
        return self.parent(new_coeffs)

    def __rsub__(self, other):
        return -self.__sub__(other)

    def __isub__(self, other):
        self = self - other
        return self

    def __mul__(self, other):
        if isinstance(other, type(self)):
            new_coeffs = self._schoolbook_multiplication(other)
        elif isinstance(other, int):
            new_coeffs = [(c * other) % self.parent.q for c in self.coeffs]
        else:
            raise NotImplementedError(
                "Polynomials can only be multiplied by each other, or scaled by integers"
            )
        return self.parent(new_coeffs)

    def __rmul__(self, other):
        return self.__mul__(other)

    def __imul__(self, other):
        self = self * other
        return self

    def __pow__(self, n):
        if not isinstance(n, int):
            raise TypeError(
                "Exponentiation of a polynomial must be done using an integer."
            )

        # Deal with negative scalar multiplication
        if n < 0:
            raise ValueError(
                "Negative powers are not supported for elements of a Polynomial Ring"
            )
        f = self
        g = self.parent(1)
        while n > 0:
            if n % 2 == 1:
                g = g * f
            f = f * f
            n = n // 2
        return g

    def __eq__(self, other):
        if isinstance(other, type(self)):
            return self.coeffs == other.coeffs
        elif isinstance(other, int):
            if self.is_constant() and (other % self.parent.q) == self.coeffs[0]:
                return True
        return False

    def __getitem__(self, idx):
        return self.coeffs[idx]

    def __repr__(self):
        if self.is_zero():
            return "0"

        info = []
        for i, c in enumerate(self.coeffs):
            if c != 0:
                if i == 0:
                    info.append(f"{c}")
                elif i == 1:
                    if c == 1:
                        info.append("x")
                    else:
                        info.append(f"{c}*x")
                else:
                    if c == 1:
                        info.append(f"x^{i}")
                    else:
                        info.append(f"{c}*x^{i}")
        return " + ".join(info)

    def __str__(self):
        return self.__repr__()


class PolynomialRingDilithium(PolynomialRing):
    def __init__(self):
        self.q = 8380417
        self.n = 256
        self.element = PolynomialDilithium
        self.element_ntt = PolynomialDilithiumNTT

        root_of_unity = 1753
        self.ntt_zetas = [
            pow(root_of_unity, self.br(i, 8), 8380417) for i in range(256)
        ]
        self.ntt_f = pow(256, -1, 8380417)

    @staticmethod
    def br(i, k):
        bin_i = bin(i & (2**k - 1))[2:].zfill(k)
        return int(bin_i[::-1], 2)

    def sample_in_ball(self, seed, tau):
        def rejection_sample(i, xof):
            while True:
                j = xof.read(1)[0]
                if j <= i:
                    return j

        # Initialise the XOF
        xof = shake256(seed)

        # Set the first 8 bytes for the sign, and leave the rest for
        # sampling.
        sign_bytes = xof.read(8)
        sign_int = int.from_bytes(sign_bytes, "little")

        # Set the list of coeffs to be 0
        coeffs = [0 for _ in range(256)]

        # Now set tau values of coeffs to be ±1
        for i in range(256 - tau, 256):
            j = rejection_sample(i, xof)
            coeffs[i] = coeffs[j]
            coeffs[j] = 1 - 2 * (sign_int & 1)
            sign_int >>= 1

        return self(coeffs)

    def rejection_sample_ntt_poly(self, rho, i, j):
        def rejection_sample(xof):
            while True:
                j_bytes = xof.read(3)
                j = int.from_bytes(j_bytes, "little")
                j &= 0x7FFFFF
                if j < 8380417:
                    return j

        # Initialise the XOF
        seed = rho + bytes([j, i])
        xof = shake128(seed)
        coeffs = [rejection_sample(xof) for _ in range(256)]
        return self(coeffs, is_ntt=True)

    def rejection_bounded_poly(self, rho_prime, i, eta):
        def coefficient_from_half_byte(j, eta):
            """
            Rejects values until a value j < 2η is found
            """
            if eta == 2 and j < 15:
                return 2 - (j % 5)
            elif j < 9:
                assert eta == 4
                return 4 - j
            return False

        # Initialise the XOF
        seed = rho_prime + int.to_bytes(i, 2, "little")
        xof = shake256(seed)

        # Sample bytes for all n coeffs
        i = 0
        coeffs = [0 for _ in range(256)]
        while i < 256:
            # Consider two values for each byte (top and bottom four bits)
            j = xof.read(1)[0]

            c0 = coefficient_from_half_byte(j % 16, eta)
            if c0 is not False:
                coeffs[i] = c0
                i += 1

            c1 = coefficient_from_half_byte(j // 16, eta)
            if c1 is not False and i < 256:
                coeffs[i] = c1
                i += 1

        return self(coeffs)

    def sample_mask_polynomial(self, rho_prime, i, kappa, gamma_1):
        if gamma_1 == (1 << 17):
            bit_count = 18
            total_bytes = 576  # (256 * 18) / 8
        else:
            bit_count = 20
            total_bytes = 640  # (256 * 20) / 8

        # Initialise the XOF
        seed = rho_prime + int.to_bytes(kappa + i, 2, "little")
        xof_bytes = shake256(seed).read(total_bytes)
        r = int.from_bytes(xof_bytes, "little")
        mask = (1 << bit_count) - 1
        coeffs = [gamma_1 - ((r >> bit_count * i) & mask) for i in range(self.n)]

        return self(coeffs)

    def __bit_unpack(self, input_bytes, n_bits):
        if (len(input_bytes) * n_bits) % 8 != 0:
            raise ValueError(
                "Input bytes do not have a length compatible with the bit length"
            )

        r = int.from_bytes(input_bytes, "little")
        mask = (1 << n_bits) - 1
        return [(r >> n_bits * i) & mask for i in range(self.n)]

    def bit_unpack_t0(self, input_bytes):
        altered_coeffs = self.__bit_unpack(input_bytes, 13)
        coefficients = [(1 << 12) - c for c in altered_coeffs]
        return self(coefficients)

    def bit_unpack_t1(self, input_bytes):
        coefficients = self.__bit_unpack(input_bytes, 10)
        return self(coefficients)

    def bit_unpack_s(self, input_bytes, eta):
        # Level 2 and 5 parameter set
        if eta == 2:
            altered_coeffs = self.__bit_unpack(input_bytes, 3)
        # Level 3 parameter set
        else:
            assert eta == 4, f"Expected eta to be either 2 or 4, got {eta = }"
            altered_coeffs = self.__bit_unpack(input_bytes, 4)

        coefficients = [eta - c for c in altered_coeffs]
        return self(coefficients)

    def bit_unpack_w(self, input_bytes, gamma_2):
        # Level 2 parameter set
        if gamma_2 == 95232:
            coefficients = self.__bit_unpack(input_bytes, 6)
        # Level 3 and 5 parameter set
        else:
            assert gamma_2 == 261888, (
                f"Expected gamma_2 to be either (q-1)/88 or (q-1)/32, got {gamma_2 = }"
            )
            coefficients = self.__bit_unpack(input_bytes, 4)

        return self(coefficients)

    def bit_unpack_z(self, input_bytes, gamma_1):
        # Level 2 parameter set
        if gamma_1 == (1 << 17):
            altered_coeffs = self.__bit_unpack(input_bytes, 18)
        # Level 3 and 5 parameter set
        else:
            assert gamma_1 == (1 << 19), (
                f"Expected gamma_1 to be either 2^17 or 2^19, got {gamma_1 = }"
            )
            altered_coeffs = self.__bit_unpack(input_bytes, 20)

        coefficients = [gamma_1 - c for c in altered_coeffs]
        return self(coefficients)

    def __call__(self, coefficients, is_ntt=False):
        if not is_ntt:
            element = self.element
        else:
            element = self.element_ntt

        if isinstance(coefficients, int):
            return element(self, [coefficients])
        if not isinstance(coefficients, list):
            raise TypeError(
                f"Polynomials should be constructed from a list of integers, of length at most d = {256}"
            )
        return element(self, coefficients)


class PolynomialDilithium(Polynomial):
    def __init__(self, parent, coefficients):
        self.parent = parent
        self.coeffs = self._parse_coefficients(coefficients)

    def to_ntt(self):
        k, l = 0, 128
        coeffs = self.coeffs[:]
        zetas = self.parent.ntt_zetas
        while l > 0:
            start = 0
            while start < 256:
                k = k + 1
                zeta = zetas[k]
                for j in range(start, start + l):
                    t = zeta * coeffs[j + l]
                    coeffs[j + l] = coeffs[j] - t
                    coeffs[j] = coeffs[j] + t
                start = l + (j + 1)
            l >>= 1

        for j in range(256):
            coeffs[j] = coeffs[j] % 8380417

        return self.parent(coeffs, is_ntt=True)

    def from_ntt(self):
        raise TypeError(f"Polynomial is of type: {type(self)}")

    def power_2_round(self, d):
        power_2 = 1 << d
        r1_coeffs = []
        r0_coeffs = []
        for c in self.coeffs:
            r = c % self.parent.q
            r0 = reduce_mod_pm(r, power_2)
            r1_coeffs.append((r - r0) >> d)
            r0_coeffs.append(r0)

        r1_poly = self.parent(r1_coeffs)
        r0_poly = self.parent(r0_coeffs)

        return r1_poly, r0_poly

    def high_bits(self, alpha, is_ntt=False):
        coeffs = [high_bits(c, alpha, self.parent.q) for c in self.coeffs]
        return self.parent(coeffs, is_ntt=is_ntt)

    def low_bits(self, alpha, is_ntt=False):
        coeffs = [low_bits(c, alpha, self.parent.q) for c in self.coeffs]
        return self.parent(coeffs, is_ntt=is_ntt)
    def decompose(self, alpha):
        coeff_high = []
        coeff_low = []
        for c in self.coeffs:
            r1, r0 = decompose(c, alpha, self.parent.q)
            coeff_high.append(r1)
            coeff_low.append(r0)
        return self.parent(coeff_high), self.parent(coeff_low)

    def check_norm_bound(self, bound):
        return any(check_norm_bound(c, bound, self.parent.q) for c in self.coeffs)
    @staticmethod
    def __bit_pack(coeffs, n_bits, n_bytes):
        r = 0
        for c in reversed(coeffs):
            r <<= n_bits
            r |= c
        return r.to_bytes(n_bytes, "little")

    def bit_pack_t0(self):
        # 416 = 256 * 13 // 8
        altered_coeffs = [(1 << 12) - c for c in self.coeffs]
        return self.__bit_pack(altered_coeffs, 13, 416)

    def bit_pack_t1(self):
        # 320 = 256 * 10 // 8
        return self.__bit_pack(self.coeffs, 10, 320)

    def bit_pack_s(self, eta):
        altered_coeffs = [self._sub_mod_q(eta, c) for c in self.coeffs]
        # Level 2 and 5 parameter set
        if eta == 2:
            return self.__bit_pack(altered_coeffs, 3, 96)
        # Level 3 parameter set
        assert eta == 4, f"Expected eta to be either 2 or 4, got {eta = }"
        return self.__bit_pack(altered_coeffs, 4, 128)

    def bit_pack_w(self, gamma_2):
        # Level 2 parameter set
        if gamma_2 == 95232:
            return self.__bit_pack(self.coeffs, 6, 192)
        # Level 3 and 5 parameter set
        assert gamma_2 == 261888, (
            f"Expected gamma_2 to be either (q-1)/88 or (q-1)/32, got {gamma_2 = }"
        )
        return self.__bit_pack(self.coeffs, 4, 128)

    def bit_pack_z(self, gamma_1):
        altered_coeffs = [self._sub_mod_q(gamma_1, c) for c in self.coeffs]
        # Level 2 parameter set
        if gamma_1 == (1 << 17):
            return self.__bit_pack(altered_coeffs, 18, 576)
        # Level 3 and 5 parameter set
        assert gamma_1 == (1 << 19), (
            f"Expected gamma_1 to be either 2^17 or 2^19, got: {gamma_1 = }"
        )
        return self.__bit_pack(altered_coeffs, 20, 640)

    def make_hint(self, other, alpha):
        coeffs = [
            make_hint(r, z, alpha, 8380417) for r, z in zip(self.coeffs, other.coeffs)
        ]
        return self.parent(coeffs)

    def make_hint_optimised(self, other, alpha):
        coeffs = [
            make_hint_optimised(r, z, alpha, 8380417)
            for r, z in zip(self.coeffs, other.coeffs)
        ]
        return self.parent(coeffs)

    def use_hint(self, other, alpha):
        coeffs = [
            use_hint(h, r, alpha, 8380417) for h, r in zip(self.coeffs, other.coeffs)
        ]
        return self.parent(coeffs)


class PolynomialDilithiumNTT(PolynomialDilithium):
    def __init__(self, parent, coefficients):
        self.parent = parent
        self.coeffs = self._parse_coefficients(coefficients)

    def to_ntt(self):
        raise TypeError(f"Polynomial is of type: {type(self)}")

    def from_ntt(self):
        l, k = 1, 256
        coeffs = self.coeffs[:]
        zetas = self.parent.ntt_zetas
        while l < 256:
            start = 0
            while start < 256:
                k = k - 1
                zeta = -zetas[k]
                for j in range(start, start + l):
                    t = coeffs[j]
                    coeffs[j] = t + coeffs[j + l]
                    coeffs[j + l] = t - coeffs[j + l]
                    coeffs[j + l] = zeta * coeffs[j + l]
                start = j + l + 1
            l = l << 1

        for j in range(256):
            coeffs[j] = (coeffs[j] * self.parent.ntt_f) % 8380417

        return self.parent(coeffs, is_ntt=False)

    def ntt_coefficient_multiplication(self, f_coeffs, g_coeffs):
        return [(c1 * c2) % 8380417 for c1, c2 in zip(f_coeffs, g_coeffs)]

    def ntt_multiplication(self, other):
        if not isinstance(other, type(self)):
            raise ValueError

        new_coeffs = self.ntt_coefficient_multiplication(self.coeffs, other.coeffs)
        return new_coeffs

    def __add__(self, other):
        new_coeffs = self._add_(other)
        return self.parent(new_coeffs, is_ntt=True)

    def __sub__(self, other):
        new_coeffs = self._sub_(other)
        return self.parent(new_coeffs, is_ntt=True)

    def __mul__(self, other):
        if isinstance(other, type(self)):
            new_coeffs = self.ntt_multiplication(other)
        elif isinstance(other, int):
            new_coeffs = [(c * other) % 8380417 for c in self.coeffs]
        else:
            raise NotImplementedError(
                f"Polynomials can only be multiplied by each other, or scaled by integers, {type(other) = }, {type(self) = }"
            )
        return self.parent(new_coeffs, is_ntt=True)


## Class com outras Propriedades Matemáticas (Matrizes, Bit Manipulation, Hints...)

In [221]:
class Module:
    def __init__(self, ring):
        self.ring = ring
        self.matrix = Matrix

    def random_element(self, m, n):
        elements = [[self.ring.random_element() for _ in range(n)] for _ in range(m)]
        return self(elements)

    def __repr__(self):
        return f"Module over the commutative ring: {self.ring}"

    def __str__(self):
        return f"Module over the commutative ring: {self.ring}"

    def __call__(self, matrix_elements, transpose=False):
        if not isinstance(matrix_elements, list):
            raise TypeError(
                "elements of a module are matrices, built from elements of the base ring"
            )

        if isinstance(matrix_elements[0], list):
            for element_list in matrix_elements:
                if not all(isinstance(aij, self.ring.element) for aij in element_list):
                    raise TypeError(
                        f"All elements of the matrix must be elements of the ring: {self.ring}"
                    )
            return self.matrix(self, matrix_elements, transpose=transpose)

        elif isinstance(matrix_elements[0], self.ring.element):
            if not all(isinstance(aij, self.ring.element) for aij in matrix_elements):
                raise TypeError(
                    f"All elements of the matrix must be elements of the ring: {self.ring}"
                )
            return self.matrix(self, [matrix_elements], transpose=transpose)

        else:
            raise TypeError(
                "elements of a module are matrices, built from elements of the base ring"
            )

    def vector(self, elements):
        return self.matrix(self, [elements], transpose=True)


class Matrix:
    def __init__(self, parent, matrix_data, transpose=False):
        self.parent = parent
        self._data = matrix_data
        self._transpose = transpose
        if not self._check_dimensions():
            raise ValueError("Inconsistent row lengths in matrix")

    def dim(self):
        if not self._transpose:
            return len(self._data), len(self._data[0])
        else:
            return len(self._data[0]), len(self._data)

    def _check_dimensions(self):
        return len(set(map(len, self._data))) == 1

    def transpose(self):
        return self.parent(self._data, not self._transpose)

    def transpose_self(self):
        self._transpose = not self._transpose
        return

    T = property(transpose)

    def reduce_coefficients(self):
        for row in self._data:
            for ele in row:
                ele.reduce_coefficients()
        return self

    def __getitem__(self, idx):
        assert isinstance(idx, tuple) and len(idx) == 2, "Can't access individual rows"
        if not self._transpose:
            return self._data[idx[0]][idx[1]]
        else:
            return self._data[idx[1]][idx[0]]

    def __eq__(self, other):
        if self.dim() != other.dim():
            return False
        m, n = self.dim()
        return all([self[i, j] == other[i, j] for i in range(m) for j in range(n)])

    def __neg__(self):
        m, n = self.dim()
        return self.parent(
            [[-self[i, j] for j in range(n)] for i in range(m)],
            self._transpose,
        )

    def __add__(self, other):
        if not isinstance(other, type(self)):
            raise TypeError("Can only add matrices to other matrices")
        if self.parent != other.parent:
            raise TypeError("Matrices must have the same base ring")
        if self.dim() != other.dim():
            raise ValueError("Matrices are not of the same dimensions")

        m, n = self.dim()
        return self.parent(
            [[self[i, j] + other[i, j] for j in range(n)] for i in range(m)],
            False,
        )

    def __iadd__(self, other):
        self = self + other
        return self

    def __sub__(self, other):
        if not isinstance(other, type(self)):
            raise TypeError("Can only add matrices to other matrices")
        if self.parent != other.parent:
            raise TypeError("Matrices must have the same base ring")
        if self.dim() != other.dim():
            raise ValueError("Matrices are not of the same dimensions")

        m, n = self.dim()
        return self.parent(
            [[self[i, j] - other[i, j] for j in range(n)] for i in range(m)],
            False,
        )

    def __isub__(self, other):
        self = self - other
        return self

    def __matmul__(self, other):
        if not isinstance(other, type(self)):
            raise TypeError("Can only multiply matrcies with other matrices")
        if self.parent != other.parent:
            raise TypeError("Matrices must have the same base ring")

        m, n = self.dim()
        n_, l = other.dim()
        if not n == n_:
            raise ValueError("Matrices are of incompatible dimensions")

        return self.parent(
            [
                [sum(self[i, k] * other[k, j] for k in range(n)) for j in range(l)]
                for i in range(m)
            ]
        )

    def scale(self, other):
        if not (isinstance(other, self.parent.ring.element) or isinstance(other, int)):
            raise TypeError("Can only multiply elements with polynomials or integers")

        matrix = [[other * ele for ele in row] for row in self._data]
        return self.parent(matrix, transpose=self._transpose)

    def dot(self, other):
        if not isinstance(other, type(self)):
            raise TypeError("Can only perform dot product with other matrices")
        res = self.T @ other
        assert res.dim() == (1, 1)
        return res[0, 0]

    def __repr__(self):
        m, n = self.dim()

        if m == 1:
            return str(self._data[0])

        max_col_width = [max(len(str(self[i, j])) for i in range(m)) for j in range(n)]
        info = "]\n[".join(
            [
                ", ".join([f"{str(self[i, j]):>{max_col_width[j]}}" for j in range(n)])
                for i in range(m)
            ]
        )
        return f"[{info}]"


class ModuleDilithium(Module):
    def __init__(self):
        self.ring = PolynomialRingDilithium()
        self.matrix = MatrixDilithium

    def __bit_unpack(self, input_bytes, m, n, alg, packed_len, *args):
        poly_bytes = [
            input_bytes[i : i + packed_len]
            for i in range(0, len(input_bytes), packed_len)
        ]
        matrix = [
            [alg(poly_bytes[n * i + j], *args) for j in range(n)] for i in range(m)
        ]
        return self(matrix)

    def bit_unpack_t0(self, input_bytes, m, n):
        packed_len = 416
        algorithm = self.ring.bit_unpack_t0
        return self.__bit_unpack(input_bytes, m, n, algorithm, packed_len)

    def bit_unpack_t1(self, input_bytes, m, n):
        packed_len = 320
        algorithm = self.ring.bit_unpack_t1
        return self.__bit_unpack(input_bytes, m, n, algorithm, packed_len)

    def bit_unpack_s(self, input_bytes, m, n, eta):
        # Level 2 and 5 parameter set
        if eta == 2:
            packed_len = 96
        # Level 3 parameter set
        elif eta == 4:
            packed_len = 128
        else:
            raise ValueError("Expected eta to be either 2 or 4")
        algorithm = self.ring.bit_unpack_s
        return self.__bit_unpack(input_bytes, m, n, algorithm, packed_len, eta)

    def bit_unpack_w(self, input_bytes, m, n, gamma_2):
        # Level 2 parameter set
        if gamma_2 == 95232:
            packed_len = 192
        # Level 3 and 5 parameter set
        elif gamma_2 == 261888:
            packed_len = 128
        else:
            raise ValueError("Expected gamma_2 to be either (q-1)/88 or (q-1)/32")
        algorithm = self.ring.bit_unpack_w
        return self.__bit_unpack(input_bytes, m, n, algorithm, packed_len, gamma_2)

    def bit_unpack_z(self, input_bytes, m, n, gamma_1):
        # Level 2 parameter set
        if gamma_1 == (1 << 17):
            packed_len = 576
        # Level 3 and 5 parameter set
        elif gamma_1 == (1 << 19):
            packed_len = 640
        else:
            raise ValueError("Expected gamma_1 to be either 2^17 or 2^19")
        algorithm = self.ring.bit_unpack_z
        return self.__bit_unpack(input_bytes, m, n, algorithm, packed_len, gamma_1)


class MatrixDilithium(Matrix):
    def __init__(self, parent, matrix_data, transpose=False):
        super().__init__(parent, matrix_data, transpose=transpose)

    def check_norm_bound(self, bound):
        for row in self._data:
            if any(p.check_norm_bound(bound) for p in row):
                return True
        return False

    def power_2_round(self, d):
        m, n = self.dim()

        m1_elements = [[0 for _ in range(n)] for _ in range(m)]
        m0_elements = [[0 for _ in range(n)] for _ in range(m)]

        for i in range(m):
            for j in range(n):
                m1_ele, m0_ele = self[i, j].power_2_round(d)
                m1_elements[i][j] = m1_ele
                m0_elements[i][j] = m0_ele

        return self.parent(m1_elements, transpose=self._transpose), self.parent(
            m0_elements, transpose=self._transpose
        )

    def decompose(self, alpha):
        m, n = self.dim()

        m1_elements = [[0 for _ in range(n)] for _ in range(m)]
        m0_elements = [[0 for _ in range(n)] for _ in range(m)]

        for i in range(m):
            for j in range(n):
                m1_ele, m0_ele = self[i, j].decompose(alpha)
                m1_elements[i][j] = m1_ele
                m0_elements[i][j] = m0_ele

        return self.parent(m1_elements, transpose=self._transpose), self.parent(
            m0_elements, transpose=self._transpose
        )

    def __bit_pack(self, algorithm, *args):
        return b"".join(algorithm(poly, *args) for row in self._data for poly in row)

    def bit_pack_t1(self):
        algorithm = self.parent.ring.element.bit_pack_t1
        return self.__bit_pack(algorithm)

    def bit_pack_t0(self):
        algorithm = self.parent.ring.element.bit_pack_t0
        return self.__bit_pack(algorithm)

    def bit_pack_s(self, eta):
        algorithm = self.parent.ring.element.bit_pack_s
        return self.__bit_pack(algorithm, eta)

    def bit_pack_w(self, gamma_2):
        algorithm = self.parent.ring.element.bit_pack_w
        return self.__bit_pack(algorithm, gamma_2)

    def bit_pack_z(self, gamma_1):
        algorithm = self.parent.ring.element.bit_pack_z
        return self.__bit_pack(algorithm, gamma_1)

    def to_ntt(self):
        data = [[x.to_ntt() for x in row] for row in self._data]
        return self.parent(data, transpose=self._transpose)

    def from_ntt(self):
        data = [[x.from_ntt() for x in row] for row in self._data]
        return self.parent(data, transpose=self._transpose)

    def high_bits(self, alpha, is_ntt=False):
        matrix = [
            [ele.high_bits(alpha, is_ntt=is_ntt) for ele in row] for row in self._data
        ]
        return self.parent(matrix)

    def low_bits(self, alpha, is_ntt=False):
        matrix = [
            [ele.low_bits(alpha, is_ntt=is_ntt) for ele in row] for row in self._data
        ]
        return self.parent(matrix)

    def make_hint(self, other, alpha):
        matrix = [
            [p.make_hint(q, alpha) for p, q in zip(r1, r2)]
            for r1, r2 in zip(self._data, other._data)
        ]
        return self.parent(matrix)

    def make_hint_optimised(self, other, alpha):
        matrix = [
            [p.make_hint_optimised(q, alpha) for p, q in zip(r1, r2)]
            for r1, r2 in zip(self._data, other._data)
        ]
        return self.parent(matrix)

    def use_hint(self, other, alpha):
        matrix = [
            [p.use_hint(q, alpha) for p, q in zip(r1, r2)]
            for r1, r2 in zip(self._data, other._data)
        ]
        return self.parent(matrix)

    def sum_hint(self):
        return sum(c for row in self._data for p in row for c in p)


## Implementação do Dilithium

#### Definição dos Paramêtros

$ d = 13 $, $ k = 8 $, $ \ell = 7 $, $ \eta = 2 $, $ \tau = 60 $, $ \omega = 75 $, $ \gamma_1 = 2^{19} = 524288 $, $ \gamma_2 = \frac{q-1}{32} = 261888 $

In [222]:
d = 13
k = 8
l = 7
eta = 2
tau = 60
omega = 75
gamma_1 = 524288 # 2^19
gamma_2 = 261888  # (q-1)/32
beta = tau * eta

M = ModuleDilithium()
R = M.ring
random_bytes = os.urandom

## Funções Auxiliares

`func_h`
1. Gera um hash de comprimento fixo a partir de uma entrada usando a função SHAKE256.






In [228]:
def func_h(input_bytes, length):
    return shake256(input_bytes).read(length)

`expand_matrix_from_seed`
1. Expande uma seed rho para criar a matriz pública A no domínio NTT
    * Inicializa uma matriz A_data com dimensões $k$ x $l$.
    * Para cada posição (i, j), chama `rejection_sample_ntt_poly` para gerar um polinômio com coeficientes uniformemente distribuídos no domínio NTT.
    * Retorna a matriz como um objeto do tipo M (Module).

In [ ]:
def expand_matrix_from_seed(rho):
    A_data = [[0 for _ in range(l)] for _ in range(k)]
    for i in range(k):
        for j in range(l):
            A_data[i][j] = R.rejection_sample_ntt_poly(rho, i, j)
    return M(A_data)

`expand_vector_from_seed`
1. Expande uma semente $\rho$' para produzir os vetores secretos s1 e s2.

    * Gera l polinômios para s1 e k polinômios para s2 usando rejection_bounded_poly.



    * Cada polinômio tem coeficientes limitados pelo parâmetro eta (pequenos valores para segurança).



    * Retorna s1 e s2 como vetores do tipo M.vector.

In [ ]:
def expand_vector_from_seed(rho_prime):
    s1_elements = [
        R.rejection_bounded_poly(rho_prime, i, eta) for i in range(l)
    ]
    s2_elements = [
        R.rejection_bounded_poly(rho_prime, i, eta)
        for i in range(l, l + k)
    ]
    s1 = M.vector(s1_elements)
    s2 = M.vector(s2_elements)
    return s1, s2

`expand_mask_vector`
1. Gera o vetor máscara y usado no processo de assinatura.
    * Cria l polinômios usando sample_mask_polynomial, com coeficientes em um intervalo definido por gamma_1;
    * Usa rho_prime como semente e kappa como um contador para garantir unicidade;
    * Retorna o vetor y como um objeto M.vector.


In [ ]:
def expand_mask_vector(rho_prime, kappa):
    elements = [
        R.sample_mask_polynomial(rho_prime, i, kappa, gamma_1)
        for i in range(l)
    ]
    return M.vector(elements)

`pack_pk`
1. Empacota a chave pública em um formato compacto.
    * Concatena a semente rho (32 bytes) com a representação empacotada de t1 (obtida via bit_pack_t1);

    * t1 contém os bits altos do vetor t, comprimidos para eficiência.

In [ ]:
def pack_pk(rho, t1):
    return rho + t1.bit_pack_t1()

`pack_sk`
1. Empacota a chave privada em um formato compacto e seguro.

    * $\rho$ (semente da matriz A, 32 bytes),
    * $\kappa$ (semente auxiliar, 32 bytes),
    * $tr$ (hash da chave pública, 32 bytes),
    * Bytes empacotados de $s1$ e $s2$ (via `bit_pack_s`),
    * Bytes empacotados de t0 (via `bit_pack_t0`).

In [ ]:
def pack_sk(rho, K, tr, s1, s2, t0):
    s1_bytes = s1.bit_pack_s(eta)
    s2_bytes = s2.bit_pack_s(eta)
    t0_bytes = t0.bit_pack_t0()
    return rho + K + tr + s1_bytes + s2_bytes + t0_bytes

`pack_h`
1. Empacota as dicas h (hint) de forma eficiente para reduzir o tamanho da assinatura.

    * Extrai as posições dos coeficientes não-zero (1s) de cada polinômio em $h$;
    * Armazena essas posições em uma lista `packed` e calcula os offsets para reconstrução;
    * Preenche com zeros até atingir o comprimento $\omega$ e retorna como bytes.

In [ ]:
def pack_h(h):
    non_zero_positions = [
        [i for i, c in enumerate(poly.coeffs) if c == 1]
        for row in h._data
        for poly in row
    ]
    packed = []
    offsets = []
    for positions in non_zero_positions:
        packed.extend(positions)
        offsets.append(len(packed))
    padding_len = omega - offsets[-1]
    packed.extend([0 for _ in range(padding_len)])
    return bytes(packed + offsets)

`pack_sig`
1.  Empacota a assinatura em um formato compacto.

    * $\zeta$ (desafio hash, 32 bytes);
    * Bytes empacotados de $z$ (via `bit_pack_z`);
    * Os bytes de z, empacotados com z.bit_pack_z(gamma_1).

    * Os bytes de h, empacotados com pack_h(h).

In [ ]:
def pack_sig(c_tilde, z, h):
    return c_tilde + z.bit_pack_z(gamma_1) + pack_h(h)

`unpack_pk`
1. Extrair a chave pública a partir de sua representação em bytes.

    * Divide pk_bytes em duas partes:
        * $\rho$: Primeiros 32 bytes, uma semente usada para gerar a matriz A.
        * $t1\_bytes$: Bytes restantes, que contêm o vetor t1 empacotado.
    * Desempacota t1 usando M.bit_unpack_t1(t1_bytes, k, 1), retornando-o como um vetor do tipo $M$.

In [ ]:
def unpack_pk(pk_bytes):
    rho, t1_bytes = pk_bytes[:32], pk_bytes[32:]
    t1 = M.bit_unpack_t1(t1_bytes, k, 1)
    return rho, t1

`unpack_sk`

1. Desempacotar a chave privada a partir de seus bytes.

    * Verifica se o comprimento de sk_bytes é consistente com os parâmetros $\eta$, $k$ e $l$;
    * Divide sk_bytes em:
        * $\rho$, $\kappa$, $tr$: 32 bytes cada, representando semente, chave de mascaramento e hash da chave pública, respectivamente.
        * Bytes de $s1$, $s2$ e $t0$, que são vetores de polinômios;
    * Desempacota:
        * $s1$ com M.bit_unpack_s($s1_bytes$, $l$, $1$, $\eta$)
        * $s2$ com M.bit_unpack_s($s2_bytes$, $k$, $1$, $\eta$)
        * $t0$ com M.bit_unpack_s($t0_bytes$, $l$, $1$, 1)

In [ ]:
def unpack_sk(sk_bytes):
    if eta == 2:
        s_bytes = 96
    else:
        s_bytes = 128
    s1_len = s_bytes * l
    s2_len = s_bytes * k
    t0_len = 416 * k
    if len(sk_bytes) != 3 * 32 + s1_len + s2_len + t0_len:
        raise ValueError("SK packed bytes is of the wrong length")
    # Split bytes between seeds and vectors
    sk_seed_bytes, sk_vec_bytes = sk_bytes[:96], sk_bytes[96:]
    # Unpack seed bytes
    rho, K, tr = (
        sk_seed_bytes[:32],
        sk_seed_bytes[32:64],
        sk_seed_bytes[64:96],
    )
    # Unpack vector bytes
    s1_bytes = sk_vec_bytes[:s1_len]
    s2_bytes = sk_vec_bytes[s1_len : s1_len + s2_len]
    t0_bytes = sk_vec_bytes[-t0_len:]
    # Unpack bytes to vectors
    s1 = M.bit_unpack_s(s1_bytes, l, 1, eta)
    s2 = M.bit_unpack_s(s2_bytes, k, 1, eta)
    t0 = M.bit_unpack_t0(t0_bytes, k, 1)
    return rho, K, tr, s1, s2, t0

`unpack_h`
1. Reconstruir a matriz de dicas $h$ a partir de seus bytes.
    * Usa os últimos k bytes de h_bytes como offsets para identificar posições;
    * Reconstrói as posições não-zero dos polinômios a partir dos bytes anteriores.
    * Gera uma matriz de polinômios com coeficientes 0 ou 1 nas posições indicadas, retornando-a como um objeto $M$.

In [ ]:
def unpack_h(h_bytes):
    offsets = [0] + list(h_bytes[-k :])
    non_zero_positions = [
        list(h_bytes[offsets[i] : offsets[i + 1]]) for i in range(k)
    ]
    matrix = []
    for poly_non_zero in non_zero_positions:
        coeffs = [0 for _ in range(256)]
        for non_zero in poly_non_zero:
            coeffs[non_zero] = 1
        matrix.append([R(coeffs)])
    return M(matrix)

`unpack_sig`
1.  Desempacotar os componentes da assinatura a partir de seus bytes.
    * Divide $sig\_bytes$ em:
        * $\zeta$: Primeiros 32 bytes, o desafio hash.
        * $z\_bytes$: Bytes intermediários até os últimos $k + \omega$, contendo $z$ empacotado.
        * $h\_bytes$: Últimos $k + \omega$ bytes, contendo h empacotado.
    * Desempacota:
        * $z$ com M.bit_unpack_z($z\_bytes$, $l$, $1$, $\gamma_1$);
        * $h$ com unpack_h($h\_bytes$).

In [ ]:
def unpack_sig(sig_bytes):
    c_tilde = sig_bytes[:32]
    z_bytes = sig_bytes[32 : -(k + omega)]
    h_bytes = sig_bytes[-(k + omega) :]
    z = M.bit_unpack_z(z_bytes, l, 1, gamma_1)
    h = unpack_h(h_bytes)
    return c_tilde, z, h

# Implementação do Dilithium

A função `keygen`  é responsável por gerar o par de chaves pública e privada:

1. Gera uma semente aleatória $\zeta$ usando os.urandom;
2. Expande $\zeta$ com SHAKE-256 para obter $\rho$, $\rho'$ e $K$;
3. Gera a matriz $\mathbf{A}$ no domínio NTT a partir de $\rho$ usando _expand_matrix_from_seed;
4. Gera os vetores secretos $\mathbf{s}_1$ e $\mathbf{s}_2$ com coeficientes em $[- \eta, \eta]$ a partir de $\rho'$;
5. Computa $\mathbf{t} = \mathbf{A} \mathbf{s}_1 + \mathbf{s}_2$, com $\mathbf{s}_1$ no domínio NTT, e aplica power_2_round para obter $\mathbf{t}_1$ e $\mathbf{t}_0$;
6. Empacota a chave pública como $(\rho, \mathbf{t}_1)$ e a chave secreta como $(\rho, K, tr, \mathbf{s}_1, \mathbf{s}_2, \mathbf{t}_0)$, onde $tr = H(\rho \| \mathbf{t}_1)$.

In [224]:
def keygen():
    #1
    # Gerar Zeta Aleatorio (32 bytes)
    zeta = random_bytes(32)

    #2
    # Gerar a seed com Zeta
    seed_bytes = func_h(zeta, 128)
    # Separar os bytes da seed em rho, rho_prime e K
    rho, rho_prime, K = seed_bytes[:32], seed_bytes[32:96], seed_bytes[96:]

    #3
    # Gerar a matriz A ∈ R^(k x l)
    A_hat = expand_matrix_from_seed(rho)

    #4
    # Gerar os vetores de erro s1 ∈ R^l, s2 ∈ R^k
    s1, s2 = expand_vector_from_seed(rho_prime)
    s1_hat = s1.to_ntt()

    #5
    # Multiplicação de matrizes
    t = (A_hat @ s1_hat).from_ntt() + s2
    t1, t0 = t.power_2_round(d)

    #6
    # Empacotar os bytes
    pk = pack_pk(rho, t1)
    tr = func_h(pk, 32)
    sk = pack_sk(rho, K, tr, s1, s2, t0)
    return pk, sk


A função `sign` gera uma assinatura para uma mensagem $M$:

1. Desempacota a chave secreta e regenera $\mathbf{A}$ a partir de $\rho$;
2. Computa $\mu = H(tr \| M)$ e $\rho' = H(K \| \mu)$ para a versão determinística;
3. Gera o vetor de mascaramento $\mathbf{y}$ com _expand_mask_vector usando $\rho'$ e um nonce $\kappa$;
4. Computa $\mathbf{w} = \mathbf{A} \mathbf{y}$ no domínio NTT e decompõe em $\mathbf{w}_1$ e $\mathbf{w}_0$ com decompose;
5. Gera o desafio $c$ como $H(\mu \| \mathbf{w}_1)$ e computa $\mathbf{z} = \mathbf{y} + c \mathbf{s}_1$;
6. Aplica verificações de rejeição (e.g., $\|\mathbf{z}\|_\infty < \gamma_1 - \beta$) e calcula as dicas $\mathbf{h}$ com `make_hint_optimised`;
7. Empacota a assinatura como $(\hat{c}, \mathbf{z}, \mathbf{h})$.
O processo está em conformidade com a especificação.

In [225]:
def sign(sk_bytes, m):
    #1
    # Desampacota a sk e regenera a matriz A
    rho, K, tr, s1, s2, t0 = unpack_sk(sk_bytes)
    # Gerar a matriz A ∈ R^(k x l)
    A_hat = expand_matrix_from_seed(rho)

    #2
    # Gera Mu, Kappa e rho_prime
    mu = func_h(tr + m, 64)
    kappa = 0
    rho_prime = func_h(K + mu, 64)

    #3
    # Gerar o Vetor Máscara y ∈ R^l
    s1 = s1.to_ntt()
    s2 = s2.to_ntt()
    t0 = t0.to_ntt()
    alpha = gamma_2 << 1
    while True:
        y = expand_mask_vector(rho_prime, kappa)
        y_hat = y.to_ntt()
        kappa += l #nonce

        #4
        #w é o resultado da multiplicação da matriz A por y
        #Converte de volta do NTT para o domínio normal
        w = (A_hat @ y_hat).from_ntt()

        # Decompõe w em w1 (bits altos) e w0 (bits baixos) usando alpha
        w1, w0 = w.decompose(alpha)
        # Empacota w1 para o hash do desafio
        w1_bytes = w1.bit_pack_w(gamma_2)

        #5
        #c é um polinômio esparso com coeficientes em {-1, 0, 1}
        #Calculado a partir de Mu e w1, garantindo ligação com a mensagem
        c_tilde = func_h(mu + w1_bytes, 32)
        c = R.sample_in_ball(c_tilde, tau)
        c = c.to_ntt()

        #6
        # Calcular z = y + c * s1
        #z é a parte principal da assinatura
        #Adiciona o desafio c escalado por s1 a y
        z = y + (s1.scale(c)).from_ntt()

        # Verifica se z está dentro do limite de norma (gamma_1 - beta)
        # Se exceder, rejeita e tenta novamente
        if z.check_norm_bound(gamma_1 - beta):
            continue

        # Calcula w0 - c * s2 e verifica a norma
        # Garante que a assinatura não revele informações sobre s2
        w0_minus_cs2 = w0 - s2.scale(c).from_ntt()
        if w0_minus_cs2.check_norm_bound(gamma_2 - beta):
            continue

        # Calcula c * t0 e verifica a norma
        # t0 é a parte baixa de t, e sua norma também deve ser limitada
        c_t0 = t0.scale(c).from_ntt()
        if c_t0.check_norm_bound(gamma_2):
            continue

        # Calcula w0 - c * s2 + c * t0 para gerar as dicas (hints)
        # As dicas h ajudam a reconstruir w1 na verificação
        w0_minus_cs2_plus_ct0 = w0_minus_cs2 + c_t0
        h = w0_minus_cs2_plus_ct0.make_hint_optimised(w1, alpha)

        # Verifica se o número de dicas não excede omega
        # Limite imposto para eficiência e segurança
        if h.sum_hint() > omega:
            continue

        # 7
        # Se todas as verificações passarem, empacota e retorna a assinatura
        # A assinatura contém c_tilde, z e h
        return pack_sig(c_tilde, z, h)

A função `verify` verifica a validade de uma assinatura:

1. Desempacota a chave pública $(\rho, \mathbf{t}_1)$ e a assinatura $(\hat{c}, \mathbf{z}, \mathbf{h})$.
2. Verifica se o número de dicas em $\mathbf{h}$ é $\leq \omega$ e se $\|\mathbf{z}\|_\infty < \gamma_1 - \beta$.
3. Reconstrói $\mathbf{A}$ a partir de $\rho$ e computa $\mathbf{w}_1' = \text{UseHint}(\mathbf{h}, \mathbf{A} \mathbf{z} - c \mathbf{t}_1 \cdot 2^d, 2\gamma_2)$.
4. Confirma que $\hat{c} = H(\mu \| \mathbf{w}_1')$.

In [226]:
def verify(pk_bytes, m, sig_bytes):
    #1
    # Desempacota a pk e a assinatura
    rho, t1 = unpack_pk(pk_bytes)
    c_tilde, z, h = unpack_sig(sig_bytes)

    #2
    #Verificar se o comprimento de h é válido
    if h.sum_hint() > omega:
        return False

    # Verificar se z está dentro dos limites
    if z.check_norm_bound(gamma_1 - beta):
        return False
    
    #3.1
    #Reconstruir 
    A_hat = expand_matrix_from_seed(rho)
    tr = func_h(pk_bytes, 32)
    mu = func_h(tr + m, 64)
    c = R.sample_in_ball(c_tilde, tau)
    
    #3.2
    # w1 = h.make_hint_optimised
    c = c.to_ntt()
    z = z.to_ntt()
    t1 = t1.scale(1 << d)
    t1 = t1.to_ntt()
    Az_minus_ct1 = (A_hat @ z) - t1.scale(c)
    Az_minus_ct1 = Az_minus_ct1.from_ntt()
    w_prime = h.use_hint(Az_minus_ct1, 2 * gamma_2)
    w_prime_bytes = w_prime.bit_pack_w(gamma_2)

    #4
    # Confirmar se c_tilde é igual a func_h(mu + w_prime_bytes, 32)
    return c_tilde == func_h(mu + w_prime_bytes, 32)

### Run

In [227]:
print ("======= Dilithium ======= \n")
pk, sk = keygen()
print(f"Chave Pública: {pk.hex()}")
print(f"Chave Privada: {sk.hex()}")
msg = b"Mensagem em Dilithium"

print(f"\nMensagem: {msg}")
sig = sign(sk, msg)
print(f"\nAssinatura: {sig.hex()}")

print("\nVerificação?", verify(pk, msg, sig))


======= Dilithium ======= 

Chave Pública: 4130977e9fb76947be0b439925443004fc3adbc22a5624ae46c69daa33d7f33a9663124433825d4d155e046e9ca4e4d31c280bb77d36f544f228602fda1fdd3be5c7d6df20e718bda040ccfcb4ef19cdb41accb36d647b249c9a8b777fd023e8653fdafbbaaa81d1837b98d33ac390530a38b1a69550393c274bef514268c845d2ad200e4304b5e9da39fb4051bc0d59625d732941e12ef9ac3a80c0ebf2812f7cfb91d0c7be4949b9acdce9d4658b111ade58a3d142cd24e759892500fd11a6b868633e78cc5d3f77e552d3988d084a4973d8f98f56ffe32249fce8775fdcfa0302a3c4dd298509157add2eab4f454e6e26474642d45dea48d2f68111d611d35705754ff228f1a1d89094651c8b8092727b1c559c522ccb568ce3cda2726e9f69e167ea4653738a0a30f2ccc0920793c31127eed54aef7482fced917d9b699e352ffd777fabd54e2f0ad4701823c41c66315fbd24a8920434a860a6fc99ff1845c915080e90915a10b3327779547c247fd372d46aa52f8b30cef255e311d2d7fa1b27cbbf17a0401480f70379f45d0ad5ddc62b049f306ec3e64561d6d13dceca22ac070a6492689e252c7c4a09fb52b16f7f9a0aa001feb1135e62ea1b0f6d218f5dc2ebc468945c3c7a995fdd653be4a64e6523873b3f4eee52187ac66